# Scenario 03: A2A Agent-to-Agent Protocol

**Estimated Time**: 45 minutes

## Learning Objectives
- Understand A2A protocol for agent interoperability
- Create Agent Cards for capability discovery
- Build A2A server and client implementations
- Enable cross-agent communication

## Prerequisites
- Completed Scenario 01 (Simple Agent + MCP)
- Understanding of REST APIs and JSON-RPC

## Part 1: Understanding A2A Protocol

### What is A2A?

A2A (Agent-to-Agent) is a protocol that enables agents to:
- **Discover** each other via Agent Cards
- **Invoke** capabilities remotely via JSON-RPC
- **Track** task progress and status
- **Exchange** structured messages and artifacts

### Key Concepts

1. **Agent Card**: Metadata describing agent capabilities (`/.well-known/agent-card.json`)
2. **Skills**: Specific capabilities an agent exposes
3. **Tasks**: Work items that agents process
4. **JSON-RPC**: Communication protocol for method invocation

### Task States

```
submitted → working → completed
                   → failed
                   → cancelled
         → input-required → ...
```

## Part 2: Setting Up the Environment

In [ ]:
# Verify imports
import sys
import json
from pathlib import Path

# Ensure we can import from src
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import A2A components
from src.agents import (
    A2AServer,
    create_a2a_server,
    AgentCard,
    Skill,
    TaskState,
    Task,
)
from src.agents.a2a_server import (
    JSONRPCRequest,
    JSONRPCResponse,
    Message,
    TextPart,
    TaskManager,
)
from src.common.telemetry import setup_telemetry, get_tracer

# Setup telemetry
setup_telemetry()
tracer = get_tracer(__name__)

print("✅ A2A components imported successfully!")
print(f"\nTask states: {[s.value for s in TaskState]}")

## Part 3: Understanding Agent Cards

Agent Cards are JSON documents that advertise an agent's capabilities.
They are served at `/.well-known/agent-card.json`.

In [ ]:
# Create an Agent Card
agent_card = AgentCard(
    name="Research Agent",
    description="Researches topics and provides summaries with sources",
    url="http://localhost:8000",
    version="1.0.0",
    skills=[
        Skill(
            id="research_topic",
            name="Research Topic",
            description="Research a topic and return findings",
            inputSchema={
                "type": "object",
                "properties": {
                    "topic": {"type": "string"},
                    "depth": {
                        "type": "string",
                        "enum": ["brief", "detailed", "comprehensive"]
                    }
                },
                "required": ["topic"]
            }
        ),
        Skill(
            id="summarize",
            name="Summarize",
            description="Summarize a document or text",
            inputSchema={
                "type": "object",
                "properties": {
                    "text": {"type": "string"},
                    "max_length": {"type": "integer"}
                },
                "required": ["text"]
            }
        )
    ]
)

print("=== Agent Card ===")
print(json.dumps(agent_card.model_dump(), indent=2))

In [ ]:
# Inspect capabilities
print("=== Agent Capabilities ===")
print(f"Streaming: {agent_card.capabilities.streaming}")
print(f"Push Notifications: {agent_card.capabilities.pushNotifications}")
print(f"State Management: {agent_card.capabilities.stateManagement}")

print("\n=== Agent Skills ===")
for skill in agent_card.skills:
    print(f"  - {skill.name} ({skill.id})")
    print(f"    {skill.description}")

## Part 4: Creating an A2A Server

In [ ]:
# Import FastAPI testing utilities
from fastapi.testclient import TestClient

# Create A2A server with skills
app = create_a2a_server(
    name="Demo Agent",
    description="Demo A2A agent for workshop",
    url="http://localhost:8000",
    skills=[
        Skill(
            id="echo",
            name="Echo",
            description="Echo back the message",
        )
    ]
)

client = TestClient(app)

# Test health endpoint
response = client.get("/health")
print(f"Health: {response.json()}")

In [ ]:
# Fetch Agent Card
response = client.get("/.well-known/agent-card.json")
card = response.json()

print("=== Retrieved Agent Card ===")
print(f"Name: {card['name']}")
print(f"URL: {card['url']}")
print(f"Skills: {[s['name'] for s in card['skills']]}")

## Part 5: JSON-RPC Communication

A2A uses JSON-RPC 2.0 for method invocation. Let's explore the protocol.

In [ ]:
# Create a JSON-RPC request
rpc_request = JSONRPCRequest(
    id="req-001",
    method="message/send",
    params={
        "message": {
            "role": "user",
            "parts": [{"kind": "text", "text": "Hello, A2A agent!"}],
            "messageId": "msg-001"
        },
        "contextId": "ctx-demo"
    }
)

print("=== JSON-RPC Request ===")
print(json.dumps(rpc_request.model_dump(), indent=2))

In [ ]:
# Send message to agent
response = client.post("/", json=rpc_request.model_dump())
result = response.json()

print("=== JSON-RPC Response ===")
print(json.dumps(result, indent=2))

In [ ]:
# Extract task information
if "result" in result:
    task_result = result["result"]
    print("=== Task Created ===")
    print(f"Task ID: {task_result.get('id')}")
    print(f"Context ID: {task_result.get('contextId')}")
    print(f"Status: {task_result.get('status', {}).get('state')}")

## Part 6: Task Management

A2A tasks go through various states. Let's explore task management.

In [ ]:
# Create and manage tasks directly
task_manager = TaskManager()

# Create a task
message = Message(
    role="user",
    parts=[TextPart(text="Research AI agents")],
    messageId="msg-test"
)

task = task_manager.create_task(context_id="ctx-test", message=message)
print(f"Created task: {task.id}")
print(f"Status: {task.status.state.value}")

In [ ]:
# Simulate task lifecycle
print("=== Task Lifecycle ===")

# Move to working
task_manager.update_task_status(task.id, TaskState.WORKING)
updated = task_manager.get_task(task.id)
print(f"1. {updated.status.state.value}")

# Move to completed
from src.agents.a2a_server import Artifact
artifact = Artifact(
    artifactId="art-001",
    name="research_result",
    parts=[TextPart(text="AI agents are autonomous systems...")]
)
task_manager.update_task_status(
    task.id, 
    TaskState.COMPLETED,
    artifacts=[artifact]
)
updated = task_manager.get_task(task.id)
print(f"2. {updated.status.state.value}")
print(f"   Artifacts: {len(updated.artifacts)}")

In [ ]:
# Get task via JSON-RPC
get_request = JSONRPCRequest(
    id="req-002",
    method="tasks/get",
    params={"id": task.id}
)

# Note: This would work with the server if we shared the task manager
print("=== tasks/get Request ===")
print(json.dumps(get_request.model_dump(), indent=2))

In [ ]:
# List tasks
list_request = JSONRPCRequest(
    id="req-003",
    method="tasks/list",
    params={
        "contextId": "ctx-test",
        "pageSize": 10
    }
)

print("=== tasks/list Request ===")
print(json.dumps(list_request.model_dump(), indent=2))

# Using our task manager directly
tasks = task_manager.list_tasks(context_id="ctx-test")
print(f"\nTasks in context: {len(tasks)}")

## Part 7: Error Handling

A2A defines standard error codes for various failure scenarios.

In [ ]:
from src.agents.a2a_server import ErrorCode, JSONRPCError

# Standard error codes
print("=== A2A Error Codes ===")
errors = {
    "PARSE_ERROR": ErrorCode.PARSE_ERROR,
    "INVALID_REQUEST": ErrorCode.INVALID_REQUEST,
    "METHOD_NOT_FOUND": ErrorCode.METHOD_NOT_FOUND,
    "INVALID_PARAMS": ErrorCode.INVALID_PARAMS,
    "INTERNAL_ERROR": ErrorCode.INTERNAL_ERROR,
    "TASK_NOT_FOUND": ErrorCode.TASK_NOT_FOUND,
    "CONTEXT_NOT_FOUND": ErrorCode.CONTEXT_NOT_FOUND,
    "RATE_LIMITED": ErrorCode.RATE_LIMITED,
    "UNAUTHORIZED": ErrorCode.UNAUTHORIZED,
}

for name, code in errors.items():
    print(f"  {name}: {code}")

In [ ]:
# Test method not found error
invalid_request = JSONRPCRequest(
    id="req-error",
    method="unknown/method",
    params={}
)

response = client.post("/", json=invalid_request.model_dump())
result = response.json()

print("=== Error Response ===")
print(json.dumps(result, indent=2))

## Part 8: Agent Discovery Pattern

In a multi-agent system, agents discover each other via Agent Cards.

In [ ]:
async def discover_agent(base_url: str, client: TestClient) -> dict:
    """Discover agent capabilities via Agent Card."""
    response = client.get("/.well-known/agent-card.json")
    if response.status_code == 200:
        return response.json()
    raise Exception(f"Failed to discover agent: {response.status_code}")

def find_skill(card: dict, skill_id: str) -> dict | None:
    """Find a specific skill in an agent card."""
    for skill in card.get("skills", []):
        if skill["id"] == skill_id:
            return skill
    return None

# Discover agent
card = await discover_agent("http://localhost:8000", client)
print(f"Discovered: {card['name']}")

# Check for specific skill
echo_skill = find_skill(card, "echo")
if echo_skill:
    print(f"\nFound skill: {echo_skill['name']}")
    print(f"Description: {echo_skill['description']}")

## Part 9: Connecting with ResearchAgent

In [ ]:
# Import agent
from src.agents import ResearchAgent
from src.tools import search_web, calculate

# Try to create A2A server with actual agent
try:
    agent = ResearchAgent(name="research_agent")
    agent.set_tool_handlers({
        "search_web": search_web,
        "calculate": calculate,
    })
    
    server = A2AServer(
        agent=agent,
        name="Research Agent",
        description="Research agent with tools",
        skills=[
            Skill(
                id="research",
                name="Research",
                description="Research any topic"
            ),
            Skill(
                id="calculate",
                name="Calculate",
                description="Perform calculations"
            )
        ]
    )
    agent_app = server.create_app()
    print("✅ A2A server with agent created!")
except Exception as e:
    print(f"⚠️ Could not create agent: {e}")
    print("Using demo server...")
    agent_app = app

## Part 10: Edge Case - Configurable Timeouts

In [ ]:
import asyncio
from dataclasses import dataclass
from typing import Optional

@dataclass
class TimeoutConfig:
    """Configuration for A2A timeouts."""
    connect_timeout: float = 5.0
    read_timeout: float = 30.0
    task_timeout: float = 300.0  # 5 minutes

async def call_with_timeout(
    func,
    timeout: float,
    error_message: str = "Operation timed out"
):
    """Execute function with timeout."""
    try:
        return await asyncio.wait_for(func, timeout=timeout)
    except asyncio.TimeoutError:
        return JSONRPCError(
            code=ErrorCode.INTERNAL_ERROR,
            message=error_message,
            data={"timeout": timeout}
        )

# Demonstrate timeout handling
async def slow_operation():
    await asyncio.sleep(2)
    return "Completed"

# This will timeout
result = await call_with_timeout(
    slow_operation(),
    timeout=1.0,
    error_message="Task execution timed out"
)

if isinstance(result, JSONRPCError):
    print(f"❌ Timeout: {result.message}")
else:
    print(f"✅ Result: {result}")

In [ ]:
# Successful with longer timeout
result = await call_with_timeout(
    slow_operation(),
    timeout=5.0,
    error_message="Task execution timed out"
)

if isinstance(result, JSONRPCError):
    print(f"❌ Timeout: {result.message}")
else:
    print(f"✅ Result: {result}")

## Part 11: Hands-On Exercise

### Exercise: Register Agent with Discovery Service

Implement a mock discovery service that agents can register with.

In [ ]:
# Exercise: Complete this discovery service

class MockDiscoveryService:
    """Mock agent discovery service."""
    
    def __init__(self):
        self.agents: dict[str, AgentCard] = {}
    
    def register(self, agent_card: AgentCard) -> str:
        """Register an agent and return registration ID."""
        import uuid
        reg_id = f"reg-{uuid.uuid4().hex[:8]}"
        self.agents[reg_id] = agent_card
        return reg_id
    
    def unregister(self, registration_id: str) -> bool:
        """Unregister an agent."""
        if registration_id in self.agents:
            del self.agents[registration_id]
            return True
        return False
    
    def find_by_skill(self, skill_id: str) -> list[AgentCard]:
        """Find agents that have a specific skill."""
        results = []
        for card in self.agents.values():
            for skill in card.skills:
                if skill.id == skill_id:
                    results.append(card)
                    break
        return results
    
    def list_all(self) -> list[AgentCard]:
        """List all registered agents."""
        return list(self.agents.values())
    
    # TODO: Add method to find agents by capability
    # TODO: Add method to search agents by description

# Test the discovery service
discovery = MockDiscoveryService()

# Register agents
reg1 = discovery.register(AgentCard(
    name="Research Agent",
    url="http://agent1.local",
    skills=[Skill(id="research", name="Research", description="Research topics")]
))

reg2 = discovery.register(AgentCard(
    name="Calculator Agent",
    url="http://agent2.local",
    skills=[Skill(id="calculate", name="Calculate", description="Do math")]
))

print(f"Registered agents: {len(discovery.list_all())}")

# Find by skill
research_agents = discovery.find_by_skill("research")
print(f"Agents with 'research' skill: {[a.name for a in research_agents]}")

## Summary

In this scenario, you learned:

1. **A2A Protocol**: Agent-to-agent communication standard
2. **Agent Cards**: Capability discovery via `/.well-known/agent-card.json`
3. **Skills**: Defining agent capabilities with schemas
4. **JSON-RPC**: Communication protocol for method invocation
5. **Tasks**: Lifecycle management (submitted → working → completed)
6. **Error Handling**: Standard error codes and responses
7. **Timeouts**: Configurable timeout handling

## Next Steps

- **Scenario 4**: Deterministic multi-agent workflows
- Build agent discovery service
- Implement agent orchestration

## Resources

- [A2A Protocol Spec](specs/001-agentic-patterns-workshop/contracts/a2a-protocol.md)
- [JSON-RPC 2.0 Specification](https://www.jsonrpc.org/specification)
- [Agent Card Standard](https://github.com/agent-card/spec)